In [1]:
import os
import openai
import cohere

from dotenv.main import load_dotenv
import os

In [2]:
load_dotenv('/Users/patrick/Documents/Pessoal/Github/llm-with-oreilly/.env')

openai.api_key = os.getenv('OPEN_AI_KEY')
co = cohere.Client(os.getenv('COHERE_API_KEY'))

In [3]:
def test_prompt_openai(prompt, suppress=False, model='text-davinci-003', **kwargs):

    response = openai.Completion.create(
      model=model,
      prompt=prompt,
      max_tokens=256,
      **kwargs
    )
    answer = response.choices[0].text
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{answer}')
    else:
        return answer

In [4]:
def test_prompt_cohere(prompt, suppress=False, model='command-xlarge-beta', **kwargs):
    response = co.generate(
        model=model,
        prompt=prompt,
        **kwargs,
#       return_likelihoods='GENERATION'
      )
    if not suppress:
        print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{response.generations[0].text}')

In [5]:
test_prompt_openai('Translate to Portuguese.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Portuguese.

Where is the nearest restaurant?
------
RESPONSE
------


Onde é o restaurante mais próximo?


In [6]:
test_prompt_cohere('Translate to Portuguese.\n\nWhere is the nearest restaurant?')

PROMPT:
------
Translate to Portuguese.

Where is the nearest restaurant?
------
RESPONSE
------

A restaurante perto de mim.


In [7]:
test_prompt_cohere('Translate to Portuguese.\n\nEnglish: Where is the nearest restaurant?')

PROMPT:
------
Translate to Portuguese.

English: Where is the nearest restaurant?
------
RESPONSE
------

Portuguese: Onde é o restaurante mais próximo?


# Language models are few-shot learners

In [8]:
examples = [
    ('Review: This movie sucks\nSubjective: Yes'),
    ('Review: This tv show was about the ocean\nSubjective: No'),
    ('Review: This book had a lot of flaws\nSubjective: Yes'),
    
    ('Review: The book was about WWII\nSubjective:'),
]

test_prompt_openai('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No


In [9]:
# Cohere is not getting this example right (actually it gets!)
test_prompt_cohere('\n###\n'.join(examples))  # ### is a common few-shot separator

PROMPT:
------
Review: This movie sucks
Subjective: Yes
###
Review: This tv show was about the ocean
Subjective: No
###
Review: This book had a lot of flaws
Subjective: Yes
###
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No


In [10]:
# Without the examples:
test_prompt_openai('Review: The book was about WWII\nSubjective:')

PROMPT:
------
Review: The book was about WWII
Subjective:
------
RESPONSE
------
 The book was an emotional roller coaster.


In [11]:
# With a prompt
test_prompt_openai('Tell me the subjectivity of this review.\n\nReview: The book was about WWII\nSubjective:')

PROMPT:
------
Tell me the subjectivity of this review.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
 This review does not contain any subjective elements.


In [12]:
# Be more specific about the output
test_prompt_openai("""Tell me the subjectivity of this review with either "Yes" or "No".

Review: The book was about WWII
Subjective:""")

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No


In [13]:
input="""Tell me the subjectivity of this review with either "Yes" or "No".

Review: The fight scenes were the best part!
Subjective:"""

test_prompt_openai(input)

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No".

Review: The fight scenes were the best part!
Subjective:
------
RESPONSE
------
 Yes


In [14]:
input="""Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.

Review: The book was about WWII
Subjective:"""

test_prompt_openai(input)

PROMPT:
------
Tell me the subjectivity of this review with either "Yes" or "No". Also as a JSON.

Review: The book was about WWII
Subjective:
------
RESPONSE
------
 No

{
  "Subjective": "No"
}


## Agent style

In [15]:
style = 'rude'

input=f"""Respond to the customer as a {style} customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:"""

test_prompt_openai(input)

PROMPT:
------
Respond to the customer as a rude customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
 Yes, I can help. But if you would have paid attention when you set up your account, you wouldn't be in this situation.


In [16]:
style = 'friendly'

input=f"""Respond to the customer as a {style} customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:"""

test_prompt_openai(input)

PROMPT:
------
Respond to the customer as a friendly customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
 Hi there! Absolutely, I'm here to help. Can you tell me the email address attached to your account?


In [17]:
style = 'yoda'

input=f"""Respond to the customer as a {style} customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:"""

test_prompt_openai(input)

PROMPT:
------
Respond to the customer as a yoda customer service agent.

Customer: Hey! I cannot seem to get into my account. Can you help?
Agent:
------
RESPONSE
------
 Help, can I? In your account get, hmmm?


# Variables in prompts

## Temperature

`Temperature = 0` means more consistency

`Temperature = 1` means more criativity

In [19]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=0,
        suppress=True
    ))

# only 3 unique responses
responses, len(set(responses))

100%|██████████| 10/10 [00:17<00:00,  1.77s/it]


([" Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help you get into your 

In [20]:
style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        suppress=True
    ))

# only 3 unique responses
responses, len(set(responses))

100%|██████████| 10/10 [00:14<00:00,  1.42s/it]


([" Hi there! Absolutely, I'm here to help. Could you please tell me your email address that you use to sign in so that I can take a closer look?",
  ' Hi there, of course! What is the issue you are having when trying to access your account?',
  " Hi there! I'd be happy to help you with that. Can you tell me what type of account it is?",
  ' Hi there, I would be more than happy to help you with getting into your account. Could you please tell me your username and I can help you reset your password?',
  " Hi there! Yes, I'm happy to help. Please can you tell me your username or email address associated with the account so that I can take a look?",
  " Of course, I'd be more than happy to help. Can you tell me what type of account you are trying to access?",
  " Hi there, sure! We'd be delighted to help you out. Can you please provide us with the email address associated with your account?",
  " Sure! I'll be happy to help you. Can you tell me a little bit more about the issue you're hav

## Top P

Top P near 0 means fewer options

In [21]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        top_p=.1,

        suppress=True
    ))
# restricting top p allows fewer tokens to be considered, making the model more deterministic
responses, len(set(responses))

100%|██████████| 10/10 [00:15<00:00,  1.55s/it]


([" Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help you get into your account. Can you tell me what type of account it is and what issue you're having?",
  " Hi there! I'd be happy to help. Can you tell me what type of account you are trying to access?",
  " Hi there! I'd be happy to help you get into your account. 

In [22]:
from tqdm import tqdm

style = 'friendly'
responses = []
for _ in tqdm(range(10)):
    responses.append(test_prompt_openai(
        f'Respond to the customer as a {style} customer service agent.\n\nCustomer: Hey! I cannot seem to get into my account. Can you help?\nAgent:',
        temperature=1,
        top_p=1,

        suppress=True
    ))
# restricting top p allows fewer tokens to be considered, making the model more deterministic
responses, len(set(responses))

100%|██████████| 10/10 [00:13<00:00,  1.36s/it]


([" Hi there! I'd be happy to help. Could you please tell me the email address associated with your account, so that we can take a look?",
  " Absolutely! I'm sorry you're having trouble logging into your account. Can I please get your username or email associated with the account so I can look into this further for you?",
  " Hi there, I'd be happy to help. Can you please verify your account details so that I can look into the issue for you?",
  " Hi there. I'd be more than happy to help you with that. Could you please tell me what email address you used to set up your account?",
  " Absolutely! Let's see what we can do. Can you please tell me what happens when you try to log in?",
  " Absolutely! I'd be happy to help you get into your account. What is the username or email address you are using?",
  " Hi there! I'm sorry to hear that you are having difficulty with your account. I'd be more than happy to help you out. Could you provide me with some additional details so I can better u

# Advanced Prompt

In [ ]:
def test_prompt_openai(prompt, suppress=False, model='text-davinci-003', **kwargs):
    
    if model in ('gpt-3.5-turbo', 'gpt-4'):
        response = openai.ChatCompletion.create(
            model=model,
            messages=[{'role': 'user', 'content': prompt}]
        ).choices[0].message.content.strip()
        if not suppress:
            print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{prompt}\n{response}')
    else:
        response = openai.Completion.create(
              model=model,
              prompt=prompt,
              max_tokens=256,
              **kwargs
            ).choices[0].text
        if not suppress:
            print(f'PROMPT:\n------\n{prompt}\n------\nRESPONSE\n------\n{prompt}{response}')
    return response